In [1]:
import os, shutil
from skimage.transform import resize 
import numpy as np
from matplotlib import pyplot
from keras.utils import to_categorical
import keras
from keras import layers
from keras import models

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

from keras import backend as k

k.clear_session()

Using TensorFlow backend.


In [2]:
def data():
    def process_data(files,folder):
        from skimage import data
        label_dict={'1':'0','2':'1','3':'2','4':'3','5':'4','6':'5','7':'6','8':'7','9':'8','10':'9','11':'10'}
        x=[]
        y=[]
        for file in files:
            limg= data.imread("Dataset/"+folder+'/'+file)
            key = file.split('_')[-1].split('.')[0]
            label_name = label_dict[key]
        #img.resize(200,200)
            y.append(label_name)
            x.append(limg)
        return x,y    
    path, dirs, files = next(os.walk("Dataset/train"))
    path, dirs2, files2 = next(os.walk("Dataset/test"))
    path, dirs3, files3 = next(os.walk("Dataset/validation"))
    
    rx_train, ry_train = process_data(files,'train')
    rx_test, ry_test = process_data(files2,'test')
    rx_val, ry_val = process_data(files3,'validation')


    x_train=np.array(rx_train).reshape(len(rx_train),34,34,1)
    x_test=np.array(rx_test).reshape(len(rx_test),34,34,1)
    x_val=np.array(rx_val).reshape(len(rx_val),34,34,1)

    X_train = np.array(x_train).astype('float32') / 255
    X_test = np.array(x_test).astype('float32') / 255
    X_val = np.array(x_val).astype('float32') / 255
    Y_train = to_categorical(ry_train)
    Y_test = to_categorical(ry_test)
    Y_val = to_categorical(ry_val)
    return X_train, Y_train, X_val, Y_val,X_test,Y_test




In [ ]:
#read image data


In [3]:
def model(X_train, Y_train, X_val, Y_val):
    
    model = models.Sequential()
    model_choice = {{choice(['one', 'two'])}}
    if model_choice == 'one':
        model.add(layers.Conv2D(16, kernel_size=3, activation='relu',padding='same', input_shape=(34,34,1)))
        model.add(layers.Conv2D(16, kernel_size=3, activation='relu',padding='same'))
        model.add(layers.MaxPooling2D(pool_size=2,strides=2))
        model.add(layers.Dropout({{uniform(0, 1)}}))

        model.add(layers.Conv2D(32, kernel_size=3, activation='relu'))
        model.add(layers.Conv2D(32, kernel_size=3, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=2,strides=2))
        model.add(layers.Dropout({{uniform(0, 1)}}))
    elif model_choice == 'two':
        model.add(layers.Conv2D(32, kernel_size=3, activation='relu',padding='same', input_shape=(34,34,1)))
        model.add(layers.Conv2D(32, kernel_size=3, activation='relu',padding='same'))
        model.add(layers.MaxPooling2D(pool_size=2,strides=2))
        model.add(layers.Dropout({{uniform(0, 1)}}))

        model.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
        model.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=2,strides=2))
        model.add(layers.Dropout({{uniform(0, 1)}}))
    
    model.add(layers.Flatten())
    model.add(layers.Dense({{choice([64, 128, 256])}}, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout({{uniform(0, 1)}}))
    choiceval = {{choice(['one', 'two'])}}
    if choiceval == 'two':
        model.add(layers.Dense({{choice([64, 128, 256])}}, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout({{uniform(0, 1)}}))
    
    model.add(layers.Dense(11, activation='softmax'))
    
    adam = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=adam)

    model.fit(X_train, Y_train,
              batch_size=128,
              nb_epoch=20,
              verbose=2,
              validation_data=(X_val, Y_val))
    score, acc = model.evaluate(X_val, Y_val, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [ ]:
X_train,Y_train, X_val, Y_val,X_test,Y_test = data()

best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='OptimizingCNN')

>>> Imports:
#coding=utf-8

try:
    import os, shutil
except:
    pass

try:
    from skimage.transform import resize
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras import layers
except:
    pass

try:
    from keras import models
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras import backend as k
except:
    pass

try:
    from skimage import data
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'model_choice': hp.choice('model_choice', ['one', 'two']),
        'Dropout': hp.uniform('Dropout', 0, 1),
        

C:\Users\ovid2\Documents\AI\temp_model.py:158: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 13200 samples, validate on 4294 samples
Epoch 1/20
 - 61s - loss: 2.4966 - acc: 0.3656 - val_loss: 1.4063 - val_acc: 0.5070
Epoch 2/20
 - 64s - loss: 1.5779 - acc: 0.4985 - val_loss: 1.2542 - val_acc: 0.5638
Epoch 3/20
 - 63s - loss: 1.3012 - acc: 0.5577 - val_loss: 1.1981 - val_acc: 0.5789
Epoch 4/20
 - 62s - loss: 1.1445 - acc: 0.5952 - val_loss: 1.1437 - val_acc: 0.5971
Epoch 5/20
 - 64s - loss: 1.0363 - acc: 0.6350 - val_loss: 1.0669 - val_acc: 0.6316
Epoch 6/20
 - 64s - loss: 0.9744 - acc: 0.6543 - val_loss: 1.0064 - val_acc: 0.6660
Epoch 7/20
 - 75s - loss: 0.8983 - acc: 0.6812 - val_loss: 1.0490 - val_acc: 0.6502
Epoch 8/20
 - 113s - loss: 0.8301 - acc: 0.7016 - val_loss: 1.0082 - val_acc: 0.6581
Epoch 9/20
 - 116s - loss: 0.7566 - acc: 0.7297 - val_loss: 0.9686 - val_acc: 0.6756
Epoch 10/20
 - 116s - loss: 0.6967 - acc: 0.7561 - val_loss: 0.7692 - val_acc: 0.7401
Epoch 11/20
 - 118s - loss: 0.6470 - acc: 0.7736 - val_loss: 0.7164 - val_acc: 0.7676
Epoch 12/20
 - 117s -

In [ ]:
print(best_run)

In [ ]:
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(34*34,)))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.59))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.66))
model.add(layers.Dense(11, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

{'Activation': 0, 'Activation_1': 0, 'Activation_2': 0, 'Dense': 3, 'Dense_1': 1, 'Dense_2': 0, 'Dropout': 0.2537673020472459, 'Dropout_1': 0.5899272287473123, 'Dropout_2': 0.6639991450877202, 'batch_size': 1, 'choiceval': 0, 'conditional': 1, 'lr': 0, 'lr_1': 1, 'lr_2': 2}

In [ ]:
#X_train,Y_train, X_val, Y_val = data()

history = model.fit(X_train,
                    Y_train,
                    epochs=20,
                    batch_size=128,
                    validation_data=(X_val, Y_val))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
print(np.argmin(val_loss)+1, np.argmax(val_acc)+1)

In [ ]:
X_train,Y_train, X_val, Y_val,X_test,Y_test = data()


In [ ]:
test_loss, test_acc = model.evaluate(X_test, Y_test,steps=7)
test_acc